In [1]:
import numpy as np
import pandas as pd
from sklearn import tree
import random
import copy

In [2]:
voice_noise_data = np.load("numpy_files/musan_mean_speech_noise.npy").item()
voice_noise_df = pd.DataFrame.from_dict(voice_noise_data)
voice_noise_df.replace([np.inf,-np.inf,np.nan],0)
voice_noise_df.dropna()
voice_noise_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 1356 entries, 0 to 1355
Data columns (total 9 columns):
RMS                  1356 non-null int64
ZCR                  1356 non-null float64
audio                1356 non-null object
bandwidth            1356 non-null float64
nwpd                 1356 non-null float64
rse                  1353 non-null float64
spectral_centroid    1356 non-null float64
spectral_flux        1356 non-null float64
spectral_rolloff     1356 non-null float64
dtypes: float64(7), int64(1), object(1)
memory usage: 95.4+ KB


In [3]:
voice_noise_df["audio"] = voice_noise_df["audio"].astype('category')
voice_noise_df["audio"] = voice_noise_df["audio"].cat.codes
voice_noise_df.head(10)

,RMS,ZCR,audio,bandwidth,nwpd,rse,spectral_centroid,spectral_flux,spectral_rolloff
0,120,0.317919,1,2.852637e+05,-1.086645,-0.190269,535.891491,0.039433,4020.624371
1,2,0.442350,1,6.743143e+05,10.745700,-0.875232,840.023296,0.061607,3833.170732
2,13,0.464951,1,8.572244e+05,1.387262,-0.322468,572.973659,0.018836,4938.532110
3,130,0.150850,1,3.938587e+06,0.498670,-0.290313,3448.585720,0.010311,6988.571429
4,42,0.250210,1,6.657314e+05,0.925133,-0.263716,745.102831,0.014008,4590.849315
5,93,0.481068,1,7.152688e+05,-0.090272,-0.184233,738.235253,0.022868,4529.035025
6,12,0.496398,1,1.041786e+06,-0.611150,-0.405754,1783.177886,0.020107,5412.625995
7,1967,0.401090,0,2.284852e+05,-0.196838,-0.301843,527.226453,0.034232,3316.301112
8,151,0.499611,1,3.597816e+05,0.241001,-0.153348,524.060903,0.011216,4010.773810
9,5,0.500247,1,5.324209e+06,-0.130207,-0.202091,2541.108767,0.006781,7003.905325


In [13]:
TRAIN_TEST_PERCENTAGE = 0.80
data = voice_noise_df.loc[:,voice_noise_df.columns != "audio"].as_matrix()
labels = voice_noise_df["audio"].as_matrix()

full_data = np.hstack((data,np.expand_dims(labels,axis = 1)))
random.shuffle(full_data)
data = full_data[:,0:-1]
labels = full_data[:,-1]

nan_indices = np.where(np.isnan(data))
print(nan_indices)
all_indices_ = np.ones(len(data),dtype = "bool")
print(all_indices_.shape)
all_indices_[nan_indices[0]] = False
data_ = copy.deepcopy(data[all_indices_,:])
labels_ = copy.deepcopy(labels[all_indices_])
print(data_.shape)
print(np.where(np.isnan(data_)))

index_ = int(len(data_)*TRAIN_TEST_PERCENTAGE)
train_data = data_[0:index_,:]
train_labels = labels_[0:index_]
test_data = data_[index_:,:]
test_labels = labels_[index_:]

print(train_data.shape)

print(np.where(np.isnan(train_data)))

print(np.where(np.isnan(test_data)))

(array([  64,  170,  551,  971, 1052]), array([4, 4, 4, 4, 4]))
(1356,)
(1351, 8)
(array([], dtype=int64), array([], dtype=int64))
(1080, 8)
(array([], dtype=int64), array([], dtype=int64))
(array([], dtype=int64), array([], dtype=int64))


In [14]:
classifier = tree.DecisionTreeClassifier()
classifier = classifier.fit(train_data,train_labels)

In [15]:
prediction = classifier.predict(test_data)
print(np.mean(np.equal(prediction,test_labels).astype(np.float32)))

0.98524
